In [1]:
import pandas as pd

# Load datasets
customers = pd.read_csv("Customers.csv")
products = pd.read_csv("Products.csv")
transactions = pd.read_csv("Transactions.csv")

# Step 1: Clean column names to avoid hidden issues
customers.columns = customers.columns.str.strip()
products.columns = products.columns.str.strip()
transactions.columns = transactions.columns.str.strip()

# Debug: Check column names after cleaning
print("Customers Columns:", customers.columns)
print("Products Columns:", products.columns)
print("Transactions Columns:", transactions.columns)

# Step 2: Merge datasets
# Merge transactions with customers and products
merged_data = pd.merge(transactions, customers, on="CustomerID", how="inner")
merged_data = pd.merge(merged_data, products, on="ProductID", how="inner")

# Debug: Display merged data columns and sample
print("Merged Data Columns:", merged_data.columns)
print("Sample Merged Data:", merged_data.head())

# Step 3: Check if required columns exist
required_columns = ["CustomerID", "TotalValue", "Quantity", "Region"]
missing_columns = [col for col in required_columns if col not in merged_data.columns]
if missing_columns:
    raise KeyError(f"The following required columns are missing: {missing_columns}")

# Step 4: Aggregate transaction data at the customer level
customer_profile = (
    merged_data.groupby("CustomerID")
    .agg(
        TotalSpend=("TotalValue", "sum"),   # Total spend per customer
        TotalQuantity=("Quantity", "sum"), # Total quantity purchased
        Region=("Region", "first")         # Region of the customer
    )
    .reset_index()
)

# Debug: Check the aggregated customer profile
print("Customer Profile (Aggregated Data):")
print(customer_profile.head())

# Step 5: One-hot encode the Region column
if "Region" in customer_profile.columns:
    customer_profile = pd.get_dummies(customer_profile, columns=["Region"], drop_first=True)

# Debug: Final customer profile structure
print("Customer Profile with Encoded Region:")
print(customer_profile.head())

# Step 6: Save the final output
customer_profile.to_csv("Sumanth_Kulkarni_Lookalike.csv", index=False)
print("Customer profile saved successfully as 'Sumanth_Kulkarni_Lookalike'")


Customers Columns: Index(['CustomerID', 'CustomerName', 'Region', 'SignupDate'], dtype='object')
Products Columns: Index(['ProductID', 'ProductName', 'Category', 'Price'], dtype='object')
Transactions Columns: Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'Price'],
      dtype='object')
Merged Data Columns: Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'Price_x', 'CustomerName', 'Region',
       'SignupDate', 'ProductName', 'Category', 'Price_y'],
      dtype='object')
Sample Merged Data:   TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067 